## ПАРСИМ AUTO.RU ДЛЯ ПРОЕКТА "[SF-DST] Car Price prediction"
https://www.kaggle.com/c/sf-dst-car-price

# ЧАСТЬ 1.
# Для начала разберёмся какие машины есть на auto.ru

Эта информация есть по ссылке https://auto.ru/htmlsitemap/mark_model_catalog.html

![2020-09-10_16-28-07.png](src/Auto_ru_parsing_10092020/2020-09-10_16-28-07.png)
#### Рис. 1

![2020-09-10_16-26-12.png](src/Auto_ru_parsing_10092020/2020-09-10_16-26-12.png)
#### Рис. 2

# Вытащим с этой страницы марки авто и их модели

In [37]:
def find_mark_and_model():
    '''
    Функция парсит инфу по маркам машин и моделям им соответсвующим c auto.ru.
    Результат возвращается грязный (с html разметкой)
    Пример результата (результат типа list):
    
    [<a href="/catalog/cars/ac/378_gt/">AC 378 GT Zagato</a>,
     <a href="/catalog/cars/ac/ace/">AC Ace</a>]
    '''
    from bs4 import BeautifulSoup
    import requests

    # скачаем страницу с марками и моделями авто
    a = requests.get('https://auto.ru/htmlsitemap/mark_model_catalog.html')
    a = a.text

    # покапаемся в этой страничке с помощью прекрасного супа
    soup = BeautifulSoup(a, 'html.parser')

    # через инспектор кода мы увидели, что интересующая нас информация находиться
    # в разметке <'div', class_ = 'sitemap'> и далее в <'a'>
    result = soup.find('div', class_ = 'sitemap').find_all('a')
    
    return result

In [38]:
def clean_info_mark_model(result):
    '''
    Эта функция работает с результатом функции find_mark_and_model().
    Смысл в том, чтобы очистить инфу от мусора и на выходе выдать словарь
    с марками и моделями авто.
    '''
    mark_model = {} # тут будет храниться наша инфа 'марка-модель'

    for i in result:
        #если такая марка авто уже есть в нашем списке то добавим к ней текущую модель
        if i.get('href').split('/')[3].upper() in mark_model:
            mark_model[i.get('href').split('/')[3].upper()] += [i.get('href').split('/')[4].upper()]

        # если нет, то создаём новую марку в нашем словаре и её мервую модель
        else:
            mark_model[i.get('href').split('/')[3].upper()] = [i.get('href').split('/')[4].upper()]

    # Результат
    num_of_mark = len(mark_model.keys())
    num_of_model = 0
    for i in mark_model.values():
        num_of_model += len(i)
    print('Количество марок авто: {0}, количество моделей авто {1}'.format(num_of_mark, num_of_model))
    
    return mark_model

In [39]:
# mark_model = clean_info_mark_model(result = find_mark_and_model())

In [40]:
# Запишем результат нашей работы в файл
def put_this_in_data(info, file_name = 'mark_model.data'):
    '''
    Эта фунция записывает переданную ей информацию в файл .data
    Данная функция является логичным продолжением функций clean_info_mark_model(result = find_mark_and_model())
    '''
    import pickle as p
    with open(file_name, 'wb') as f:
       p.dump(info, f)
    print('Файл успешно записан в текущей директории под именем:', file_name)
    return

#put_this_in_data(info = mark_model, file_name = 'mark_model.data')

# Часть 2.
# Парсим объявления с auto.ru

### Парсинг выполнялся с помощью библиотеки requests
Через инспектор коды был найден подгружаемый файл, который нас интересует
Файл подгружается по запросу. Корректный запрос содержит url, заголовки (просто копируем из инпектора) и фильтр запроса.

![2020-09-10_17-46-40.png](src/Auto_ru_parsing_10092020/2020-09-10_17-46-40.png)

![2020-09-10_17-53-16.png](src/Auto_ru_parsing_10092020/2020-09-10_17-53-16.png)

- Вся нужная нам информация подгружается в видей файла json (будем использовать соотвествующую библиотеку). 
- По указанному в запросе филтру выводится результат поиска на нескольких страницах.
- Страницу нужно указывать в запросе. Страниц не может быть более 99. Объявление на 1 странице не более 37.
- Таким образом информацию нужно выуживать пачками по не более чем 3 663 объявления. 
- Запросы нужно разбить так, чтобы в результате поиска объявлений было не более этого числа (иначе не избежать потери информации).
- Также результаты поиска не должны дублироваться в разных запросах.

### Будем разбивать запросы по регионам и моделям авто

In [41]:
import pandas as pd
import pickle as p
import requests, json
import pprint as pp
from time import sleep

In [42]:
# подгружаем файл с марками и моделями машин (пригодиться для формирования аргумента для запроса)
with open('mark_model.data', 'rb') as f:
    mark_model = p.load(f)

In [43]:
#mark_model['BMW']

In [44]:
all_cars = list(mark_model.keys())
print(' Всего марок авто:', len(all_cars),'\n Примеры: ', all_cars[0:267:30])

 Всего марок авто: 267 
 Примеры:  ['AC', 'BRISTOL', 'DELAGE', 'GORDON', 'JINBEI', 'MERCEDES', 'PUMA', 'SPYKER', 'WESTFIELD']


### Выкачивать все марки авто кажется неразумным. Ограничимся некоторым списком

In [45]:
popular_cars = ['ALFA_ROMEO', 'ASTON_MARTIN', 'AUDI', 'AURUS', 
             'BENTLEY', 'BRABUS', 'BUGATTI', 'BUICK', 
             'CADILLAC', 'CHERY', 'CHEVROLET', 'CHRYSLER', 'CITROEN',
             'DAEWOO', 'DAIHATSU', 'DATSUN', 'DELOREAN', 'DODGE', 'DW_HOWER',
             'FERRARI', 'FIAT', 'FORD', 'GEELY', 'GENESIS',
             'HAVAL', 'HONDA', 'HUMMER', 'HYUNDAI', 
             'INFINITI', 'ISUZU', 'IVECO', 'JAGUAR', 'JEEP', 'KIA', 'LOTUS', 
             'MARLIN', 'MARUSSIA', 'MASERATI', 'MAYBACH', 'MAZDA', 'MCLAREN', 'MERCEDES', 'MINI', 'MITSUBISHI',
             'NISSAN', 'OPEL', 'PAGANI', 'PEUGEOT', 'PONTIAC', 'PORSCHE', 'RENAULT', 'ROLLS_ROYCE', 'ROVER', 
             'SAAB', 'SSANG_YONG', 'SUBARU', 'SUZUKI', 'TESLA', 'TOYOTA', 'VOLKSWAGEN', 'VOLVO']

### Auto.ru использует коды регионов, перечень которых можно найти по ссылке ниже
https://semantica.in/blog/kody-regionov-yandeksa-urok-geografii.html

### Было принято решение остановиться на следующих регионах:
- 3 Центральный федеральный округ
- 17 Северо-Западный федеральный округ
- 40 Приволжский федеральный округ
- 26 Южный федеральный округ

Всего 48 областей

### Дополнительные регионы для обогащения train.
 - 52 Уральский федеральный округ
 - 102444 Северо-Кавказский федеральный округ

### Коды регионов были записаны в соответсвующий файл 'region_nums.csv'

In [67]:
# подгружаем файл с номерами регионов
region_nums = pd.read_csv('region_nums.csv', sep = ';')

In [74]:
region_nums.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   district_num  61 non-null     int64 
 1   district      61 non-null     object
 2   region_num    61 non-null     int64 
 3   region        61 non-null     object
dtypes: int64(2), object(2)
memory usage: 2.0+ KB


In [48]:
region_nums.district_num.value_counts().index

Int64Index([3, 40, 17, 102444, 26, 52], dtype='int64')

In [49]:
# Эти две штуки нужны как аргументы к запросу. Они постоянны и без них низя
url = "https://auto.ru/-/ajax/desktop/listing/"
headers = {
            'Accept': '*/*',
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
            'Connection': 'keep-alive',
            'Content-Length': '77',
            'content-type': 'application/json',
            'Cookie': '_csrf_token=23608122d7607cf614fa3696f170304f2c942825086d7100; autoru_sid=a%3Ag5e54cb202kbrp94k5stjgeuok5qc0lr.a4bf836f5de7a860384d4bb97a55d0e0%7C1582615328026.604800.Q6xoprzzbkFU9J433uQN0A.34nVWWhPYMf2gQUyYdxm-hAPmkz-UZuRfIKcHxDth7w; autoruuid=g5e54cb202kbrp94k5stjgeuok5qc0lr.a4bf836f5de7a860384d4bb97a55d0e0; suid=f70cb3a665b30dabcc2aefb60beba688.092164db5081dbf54d3645dd1160a738; yandexuid=4412548211582615329; _ym_wasSynced=%7B%22time%22%3A1582615332141%2C%22params%22%3A%7B%22eu%22%3A0%7D%2C%22bkParams%22%3A%7B%7D%7D; gdpr=0; _ym_uid=1582615332878294126; _ym_isad=2; gids=; from=direct; X-Vertis-DC=myt; _ym_visorc_22753222=b; cycada=1Ra6P74hpA1DJTHrQBDvqC2RWMZ0zfqbWbqR95lvoFc=; from_lifetime=1582636077325; _ym_d=1582636077',
            'Host': 'auto.ru',
            'Origin': 'https://auto.ru',
            'Referer': 'https://auto.ru/cars/used/?sort=fresh_relevance_1-desc&page=1',
            'Sec-Fetch-Dest': 'empty',
            'Sec-Fetch-Mode': 'same-origin',
            'Sec-Fetch-Site': 'same-origin',
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.116 Safari/537.36',
            'x-client-app-version': '202002.20.164944',
            'x-client-date': '1582636185640',
            'x-csrf-token': '23608122d7607cf614fa3696f170304f2c942825086d7100',
            'x-page-request-id': '4e70327218c7ba488485edc641875179',
            'x-requested-with': 'fetch'
}

In [77]:
def read_offers(auto_ru, data, id_, page, mark, model, too_popular_cars, district, region):
    '''
    Эта функция производит работу по ответу запроса.
    Она выковыривает из объявлений нужную нам информацию 
    и записывает (или дозаписывает) её в словарь auto_ru
    '''
    for i in data['offers']:
        # 1. bodyType:
        try: bodyType = i['vehicle_info']['configuration']['human_name']
        except: bodyType = None
        # 2. brand
        try: brand = i['vehicle_info']['mark_info']['code']
        except: brand = None
        # 3. color
        try: color = i['color_hex']
        except: color = None
        # 4. fuelType
        try: fuelType = i['lk_summary'].split()[-1]
        except: fuelType = None
        # 5. modelDate
        try: modelDate = i['vehicle_info']['super_gen']['year_from']
        except: modelDate = None
        # 6. name
        try: name =  i['vehicle_info']['tech_param']['human_name']
        except: name = None
        # 7. numberOfDoors
        try: numberOfDoors = i['vehicle_info']['configuration']['doors_count']
        except: numberOfDoors = None
        # 8. productionDate
        try: productionDate = i['documents']['year']
        except: productionDate = None
        # 9. vehicleConfiguration
        try: transmission = i['vehicle_info']['tech_param']['transmission']
        except: transmission = None
        # 10. engineDisplacement
        try: engineDisplacement = i['vehicle_info']['tech_param']['human_name'].split()[0]
        except: engineDisplacement = None
        # 11. enginePower
        try: enginePower = i['vehicle_info']['tech_param']['power']
        except: enginePower = None
        # 12. description
        try: description = i['description']
        except: description = None
        # 13. mileage
        try: mileage = i['state']['mileage']
        except: mileage = None
        # 14. Комплектация
        try: complectation = i['vehicle_info']['complectation']
        except: complectation = None
        # 15. Привод
        try: lk_summary_WD = i['lk_summary'].split(', ')[-2]
        except: lk_summary_WD = None
        # 16. Руль
        try: steering_wheel = i['vehicle_info']['steering_wheel']
        except: steering_wheel = None
        # 17. Владельцы
        try: owners_number = i['documents']['owners_number']
        except: owners_number = None
        # 18. ПТС
        try: pts = i['documents']['pts']
        except: pts = None
        # 19. Таможня
        try: custom_cleared = i['documents']['custom_cleared']
        except: custom_cleared = None
        # 20. Владение
        try: purchase_date =  i['documents']['purchase_date']
        except: purchase_date = None
        # 21. Price
        try: Price = i['price_info']['RUR']
        except: Price = None
        auto_ru[id_] = {
            'district_id': district,
            'region_id': region,
            'bodyType': bodyType,
            'brand': brand,
            'model': model,
            'color': color,
            'fuelType': fuelType,
            'modelDate': modelDate,
            'name': name,
            'numberOfDoors': numberOfDoors,
            'productionDate': productionDate,
            'vehicleConfiguration': transmission,
            'engineDisplacement': engineDisplacement,
            'enginePower': enginePower,
            'description': description,
            'mileage': mileage,
            'Комплектация': complectation,
            'Привод': lk_summary_WD,
            'Руль': steering_wheel,
            'Владельцы': owners_number,
            'ПТС': pts,
            'Таможня': custom_cleared,
            'Владение': purchase_date,
            'Price': Price            
        }
        id_ += 1
        print(f'DataSetRows: {id_}', mark, model, f'Page: {page}')
        if page == 99:
            print('ВНИМАНИЕ! Достигнуто максимальное количество страниц!')
            print('Эта модель машины слишком популярна. РАЗБЕЙ эту выборку!')
            too_popular_cars.append([mark, model])
    return auto_ru, id_, too_popular_cars

In [71]:
region_nums[region_nums.district_num == 52]

,district_num,district,region_num,region
55,52,UFD,11158,Kurgan
56,52,UFD,11162,Sverdlovsk
57,52,UFD,11176,Tyumen
58,52,UFD,11232,Yamalo-Nenets
59,52,UFD,11193,Khanty-Mansi
60,52,UFD,11225,Chelyabinsk


In [68]:
list(region_nums[region_nums.district_num == 52]['region_num'].value_counts().index)

[11158, 11232, 11225, 11162, 11193, 11176]

In [78]:
region_dict = region_nums[['region_num', 'region']].set_index('region_num').T.to_dict('list')
district_dict = region_nums[['district_num', 'district']].set_index('district_num').T.to_dict('list')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  


In [104]:
def step_by_step():
    '''
    Основная часть парсинга. Просто запусти и иди спать :)
    '''
    names_file_list = []
    auto_ru = {}
    too_popular_cars = []

    id_ = 0

    distritcs = list(region_nums['district_num'].value_counts().index)


    for district in distritcs: # [3, 40, 17, 26]:
        regions = list(region_nums[region_nums.district_num == district]['region_num'].value_counts().index)

        for region in regions:
            print('****Parcing in', district_dict[district][0], region_dict[region][0], '***')

            for mark in ['BMW']:#popular_cars: #  # all_cars:

                for model in mark_model[mark]:
                    for page in range(1, 100):
                        param = {
                            'catalog_filter' : [{"mark": mark, "model": model}],
                            'section': "all",
                            'category': "cars",
                            'geo_id':[region],
                            'page': page
                        }

                        # Осуществляем запрос
                        i = 0
                        flag = True
                        while i<3 and flag == True:
                            try:
                                response = requests.post(url, json=param, headers = headers)
                                data = response.json()
                                flag = False                            
                            except:
                                i+=1
                                print ('Что-то пошло не так с запросом. Выполним sleep(60) и попробуем ещё раз.', i)
                                sleep(3) #sleep(60)

                        try:
                            if data['offers'] == []:
                                print('Такой машины (', mark,' ', model,') нет в ',region_dict[region][0], sep ='')
                                break
                        except:
                            print('Какая-то ошибка c "offers"...Едем дальше :)')
                            #sleep(60)
                            break

                        auto_ru, id_, too_popular_cars = read_offers(auto_ru, data, id_, page, mark, model, too_popular_cars, district, region)
    return auto_ru

### Если запустил этот шайтанпарсинг, но решил его остановить принудительно, то жми кнопку interrupt несколько раз подряд. ( одно нажатие не помешает парсить дальше :DDD )

![2020-09-10_19-17-08.png](src/Auto_ru_parsing_10092020/2020-09-10_19-17-08.png)

In [105]:
%%time
auto_ru = step_by_step()

****Parcing in CFD Ivanovo ***
Такой машины (BMW 02) нет в Ivanovo
DataSetRows: 1 BMW 1ER Page: 1
Такой машины (BMW 1ER) нет в Ivanovo
Такой машины (BMW M1) нет в Ivanovo
Такой машины (BMW 2ER) нет в Ivanovo
Такой машины (BMW 2ACTIVETOURER) нет в Ivanovo
Такой машины (BMW 2GRANDTOURER) нет в Ivanovo
Такой машины (BMW 2000_C_CS) нет в Ivanovo
DataSetRows: 2 BMW 3ER Page: 1
DataSetRows: 3 BMW 3ER Page: 1
DataSetRows: 4 BMW 3ER Page: 1
DataSetRows: 5 BMW 3ER Page: 1
DataSetRows: 6 BMW 3ER Page: 1
DataSetRows: 7 BMW 3ER Page: 1
DataSetRows: 8 BMW 3ER Page: 1
DataSetRows: 9 BMW 3ER Page: 1
DataSetRows: 10 BMW 3ER Page: 1
DataSetRows: 11 BMW 3ER Page: 1
DataSetRows: 12 BMW 3ER Page: 1
DataSetRows: 13 BMW 3ER Page: 1
DataSetRows: 14 BMW 3ER Page: 1
DataSetRows: 15 BMW 3ER Page: 1
DataSetRows: 16 BMW 3ER Page: 1
Такой машины (BMW 3ER) нет в Ivanovo
Такой машины (BMW 3_15) нет в Ivanovo
Такой машины (BMW 315) нет в Ivanovo
Такой машины (BMW 3200) нет в Ivanovo
Такой машины (BMW 321) нет в Ivano

Такой машины (BMW 5ER) нет в Voronezh
Такой машины (BMW 501) нет в Voronezh
Такой машины (BMW 502) нет в Voronezh
Такой машины (BMW 503) нет в Voronezh
Такой машины (BMW 507) нет в Voronezh
DataSetRows: 199 BMW 6ER Page: 1
DataSetRows: 200 BMW 6ER Page: 1
DataSetRows: 201 BMW 6ER Page: 1
DataSetRows: 202 BMW 6ER Page: 1
DataSetRows: 203 BMW 6ER Page: 1
DataSetRows: 204 BMW 6ER Page: 1
DataSetRows: 205 BMW 6ER Page: 1
Такой машины (BMW 6ER) нет в Voronezh
Такой машины (BMW 600) нет в Voronezh
DataSetRows: 206 BMW 7ER Page: 1
DataSetRows: 207 BMW 7ER Page: 1
DataSetRows: 208 BMW 7ER Page: 1
DataSetRows: 209 BMW 7ER Page: 1
DataSetRows: 210 BMW 7ER Page: 1
DataSetRows: 211 BMW 7ER Page: 1
DataSetRows: 212 BMW 7ER Page: 1
DataSetRows: 213 BMW 7ER Page: 1
DataSetRows: 214 BMW 7ER Page: 1
DataSetRows: 215 BMW 7ER Page: 1
DataSetRows: 216 BMW 7ER Page: 1
DataSetRows: 217 BMW 7ER Page: 1
DataSetRows: 218 BMW 7ER Page: 1
DataSetRows: 219 BMW 7ER Page: 1
DataSetRows: 220 BMW 7ER Page: 1
DataSetR

Такой машины (BMW 7ER) нет в Vladimir
Такой машины (BMW 700) нет в Vladimir
Такой машины (BMW 8ER) нет в Vladimir
Такой машины (BMW E3) нет в Vladimir
Такой машины (BMW E9) нет в Vladimir
DataSetRows: 379 BMW I3 Page: 1
Такой машины (BMW I3) нет в Vladimir
Такой машины (BMW I8) нет в Vladimir
Такой машины (BMW M2) нет в Vladimir
Такой машины (BMW M3) нет в Vladimir
Такой машины (BMW M4) нет в Vladimir
Такой машины (BMW M5) нет в Vladimir
Такой машины (BMW M6) нет в Vladimir
Такой машины (BMW M8) нет в Vladimir
Такой машины (BMW NEW_CLASS) нет в Vladimir
DataSetRows: 380 BMW X1 Page: 1
DataSetRows: 381 BMW X1 Page: 1
DataSetRows: 382 BMW X1 Page: 1
DataSetRows: 383 BMW X1 Page: 1
DataSetRows: 384 BMW X1 Page: 1
Такой машины (BMW X1) нет в Vladimir
Такой машины (BMW X2) нет в Vladimir
DataSetRows: 385 BMW X3 Page: 1
DataSetRows: 386 BMW X3 Page: 1
DataSetRows: 387 BMW X3 Page: 1
DataSetRows: 388 BMW X3 Page: 1
Такой машины (BMW X3) нет в Vladimir
Такой машины (BMW X3_M) нет в Vladimir
Та

Такой машины (BMW 2ACTIVETOURER) нет в Kaluga
Такой машины (BMW 2GRANDTOURER) нет в Kaluga
Такой машины (BMW 2000_C_CS) нет в Kaluga
DataSetRows: 534 BMW 3ER Page: 1
DataSetRows: 535 BMW 3ER Page: 1
DataSetRows: 536 BMW 3ER Page: 1
DataSetRows: 537 BMW 3ER Page: 1
DataSetRows: 538 BMW 3ER Page: 1
DataSetRows: 539 BMW 3ER Page: 1
DataSetRows: 540 BMW 3ER Page: 1
DataSetRows: 541 BMW 3ER Page: 1
DataSetRows: 542 BMW 3ER Page: 1
DataSetRows: 543 BMW 3ER Page: 1
DataSetRows: 544 BMW 3ER Page: 1
DataSetRows: 545 BMW 3ER Page: 1
DataSetRows: 546 BMW 3ER Page: 1
DataSetRows: 547 BMW 3ER Page: 1
DataSetRows: 548 BMW 3ER Page: 1
DataSetRows: 549 BMW 3ER Page: 1
DataSetRows: 550 BMW 3ER Page: 1
DataSetRows: 551 BMW 3ER Page: 1
DataSetRows: 552 BMW 3ER Page: 1
DataSetRows: 553 BMW 3ER Page: 1
Такой машины (BMW 3ER) нет в Kaluga
Такой машины (BMW 3_15) нет в Kaluga
Такой машины (BMW 315) нет в Kaluga
Такой машины (BMW 3200) нет в Kaluga
Такой машины (BMW 321) нет в Kaluga
Такой машины (BMW 326) не

Такой машины (BMW 600) нет в Tula
DataSetRows: 704 BMW 7ER Page: 1
DataSetRows: 705 BMW 7ER Page: 1
DataSetRows: 706 BMW 7ER Page: 1
Такой машины (BMW 7ER) нет в Tula
Такой машины (BMW 700) нет в Tula
DataSetRows: 707 BMW 8ER Page: 1
Такой машины (BMW 8ER) нет в Tula
Такой машины (BMW E3) нет в Tula
Такой машины (BMW E9) нет в Tula
Такой машины (BMW I3) нет в Tula
Такой машины (BMW I8) нет в Tula
Такой машины (BMW M2) нет в Tula
Такой машины (BMW M3) нет в Tula
DataSetRows: 708 BMW M4 Page: 1
Такой машины (BMW M4) нет в Tula
Такой машины (BMW M5) нет в Tula
Такой машины (BMW M6) нет в Tula
Такой машины (BMW M8) нет в Tula
Такой машины (BMW NEW_CLASS) нет в Tula
DataSetRows: 709 BMW X1 Page: 1
DataSetRows: 710 BMW X1 Page: 1
DataSetRows: 711 BMW X1 Page: 1
DataSetRows: 712 BMW X1 Page: 1
DataSetRows: 713 BMW X1 Page: 1
Такой машины (BMW X1) нет в Tula
Такой машины (BMW X2) нет в Tula
DataSetRows: 714 BMW X3 Page: 1
DataSetRows: 715 BMW X3 Page: 1
DataSetRows: 716 BMW X3 Page: 1
DataSetR

...

DataSetRows: 18469 BMW 3ER Page: 3
DataSetRows: 18470 BMW 3ER Page: 3
DataSetRows: 18471 BMW 3ER Page: 3
DataSetRows: 18472 BMW 3ER Page: 3
DataSetRows: 18473 BMW 3ER Page: 3
DataSetRows: 18474 BMW 3ER Page: 3
DataSetRows: 18475 BMW 3ER Page: 3
DataSetRows: 18476 BMW 3ER Page: 3
DataSetRows: 18477 BMW 3ER Page: 3
DataSetRows: 18478 BMW 3ER Page: 3
DataSetRows: 18479 BMW 3ER Page: 3
DataSetRows: 18480 BMW 3ER Page: 3
DataSetRows: 18481 BMW 3ER Page: 3
DataSetRows: 18482 BMW 3ER Page: 3
DataSetRows: 18483 BMW 3ER Page: 3
DataSetRows: 18484 BMW 3ER Page: 3
DataSetRows: 18485 BMW 3ER Page: 3
DataSetRows: 18486 BMW 3ER Page: 3
DataSetRows: 18487 BMW 3ER Page: 3
DataSetRows: 18488 BMW 3ER Page: 3
DataSetRows: 18489 BMW 3ER Page: 3
DataSetRows: 18490 BMW 3ER Page: 3
DataSetRows: 18491 BMW 3ER Page: 3
DataSetRows: 18492 BMW 3ER Page: 3
DataSetRows: 18493 BMW 3ER Page: 3
DataSetRows: 18494 BMW 3ER Page: 3
DataSetRows: 18495 BMW 3ER Page: 3
DataSetRows: 18496 BMW 3ER Page: 3
DataSetRows: 18497 B

DataSetRows: 18717 BMW 7ER Page: 2
Такой машины (BMW 7ER) нет в Sverdlovsk
Такой машины (BMW 700) нет в Sverdlovsk
DataSetRows: 18718 BMW 8ER Page: 1
Такой машины (BMW 8ER) нет в Sverdlovsk
Такой машины (BMW E3) нет в Sverdlovsk
Такой машины (BMW E9) нет в Sverdlovsk
Такой машины (BMW I3) нет в Sverdlovsk
Такой машины (BMW I8) нет в Sverdlovsk
Такой машины (BMW M2) нет в Sverdlovsk
DataSetRows: 18719 BMW M3 Page: 1
Такой машины (BMW M3) нет в Sverdlovsk
DataSetRows: 18720 BMW M4 Page: 1
Такой машины (BMW M4) нет в Sverdlovsk
DataSetRows: 18721 BMW M5 Page: 1
Такой машины (BMW M5) нет в Sverdlovsk
Такой машины (BMW M6) нет в Sverdlovsk
Такой машины (BMW M8) нет в Sverdlovsk
Такой машины (BMW NEW_CLASS) нет в Sverdlovsk
DataSetRows: 18722 BMW X1 Page: 1
DataSetRows: 18723 BMW X1 Page: 1
DataSetRows: 18724 BMW X1 Page: 1
DataSetRows: 18725 BMW X1 Page: 1
DataSetRows: 18726 BMW X1 Page: 1
DataSetRows: 18727 BMW X1 Page: 1
DataSetRows: 18728 BMW X1 Page: 1
DataSetRows: 18729 BMW X1 Page: 1


DataSetRows: 18940 BMW X6 Page: 2
DataSetRows: 18941 BMW X6 Page: 2
DataSetRows: 18942 BMW X6 Page: 2
DataSetRows: 18943 BMW X6 Page: 2
DataSetRows: 18944 BMW X6 Page: 2
DataSetRows: 18945 BMW X6 Page: 2
DataSetRows: 18946 BMW X6 Page: 2
DataSetRows: 18947 BMW X6 Page: 2
DataSetRows: 18948 BMW X6 Page: 2
DataSetRows: 18949 BMW X6 Page: 2
DataSetRows: 18950 BMW X6 Page: 2
DataSetRows: 18951 BMW X6 Page: 2
DataSetRows: 18952 BMW X6 Page: 2
DataSetRows: 18953 BMW X6 Page: 2
DataSetRows: 18954 BMW X6 Page: 2
DataSetRows: 18955 BMW X6 Page: 2
DataSetRows: 18956 BMW X6 Page: 2
DataSetRows: 18957 BMW X6 Page: 2
DataSetRows: 18958 BMW X6 Page: 2
DataSetRows: 18959 BMW X6 Page: 2
DataSetRows: 18960 BMW X6 Page: 2
DataSetRows: 18961 BMW X6 Page: 2
DataSetRows: 18962 BMW X6 Page: 2
DataSetRows: 18963 BMW X6 Page: 2
Такой машины (BMW X6) нет в Sverdlovsk
DataSetRows: 18964 BMW X6_M Page: 1
DataSetRows: 18965 BMW X6_M Page: 1
DataSetRows: 18966 BMW X6_M Page: 1
Такой машины (BMW X6_M) нет в Sverdlo

DataSetRows: 19142 BMW X5 Page: 2
DataSetRows: 19143 BMW X5 Page: 2
Такой машины (BMW X5) нет в Khanty-Mansi
DataSetRows: 19144 BMW X5_M Page: 1
DataSetRows: 19145 BMW X5_M Page: 1
Такой машины (BMW X5_M) нет в Khanty-Mansi
DataSetRows: 19146 BMW X6 Page: 1
DataSetRows: 19147 BMW X6 Page: 1
DataSetRows: 19148 BMW X6 Page: 1
DataSetRows: 19149 BMW X6 Page: 1
DataSetRows: 19150 BMW X6 Page: 1
DataSetRows: 19151 BMW X6 Page: 1
DataSetRows: 19152 BMW X6 Page: 1
DataSetRows: 19153 BMW X6 Page: 1
DataSetRows: 19154 BMW X6 Page: 1
DataSetRows: 19155 BMW X6 Page: 1
DataSetRows: 19156 BMW X6 Page: 1
DataSetRows: 19157 BMW X6 Page: 1
DataSetRows: 19158 BMW X6 Page: 1
DataSetRows: 19159 BMW X6 Page: 1
DataSetRows: 19160 BMW X6 Page: 1
Такой машины (BMW X6) нет в Khanty-Mansi
DataSetRows: 19161 BMW X6_M Page: 1
Такой машины (BMW X6_M) нет в Khanty-Mansi
DataSetRows: 19162 BMW X7 Page: 1
DataSetRows: 19163 BMW X7 Page: 1
DataSetRows: 19164 BMW X7 Page: 1
DataSetRows: 19165 BMW X7 Page: 1
DataSetRow

Такой машины (BMW X5) нет в Tyumen
Такой машины (BMW X5_M) нет в Tyumen
DataSetRows: 19341 BMW X6 Page: 1
DataSetRows: 19342 BMW X6 Page: 1
DataSetRows: 19343 BMW X6 Page: 1
DataSetRows: 19344 BMW X6 Page: 1
DataSetRows: 19345 BMW X6 Page: 1
DataSetRows: 19346 BMW X6 Page: 1
DataSetRows: 19347 BMW X6 Page: 1
DataSetRows: 19348 BMW X6 Page: 1
DataSetRows: 19349 BMW X6 Page: 1
DataSetRows: 19350 BMW X6 Page: 1
DataSetRows: 19351 BMW X6 Page: 1
DataSetRows: 19352 BMW X6 Page: 1
DataSetRows: 19353 BMW X6 Page: 1
DataSetRows: 19354 BMW X6 Page: 1
DataSetRows: 19355 BMW X6 Page: 1
DataSetRows: 19356 BMW X6 Page: 1
DataSetRows: 19357 BMW X6 Page: 1
DataSetRows: 19358 BMW X6 Page: 1
DataSetRows: 19359 BMW X6 Page: 1
DataSetRows: 19360 BMW X6 Page: 1
DataSetRows: 19361 BMW X6 Page: 1
DataSetRows: 19362 BMW X6 Page: 1
DataSetRows: 19363 BMW X6 Page: 1
Такой машины (BMW X6) нет в Tyumen
DataSetRows: 19364 BMW X6_M Page: 1
Такой машины (BMW X6_M) нет в Tyumen
Такой машины (BMW X7) нет в Tyumen
Так

In [107]:
auto_df = pd.DataFrame.from_dict(auto_ru, orient='index')
df = auto_df.dropna(subset=['Price'], axis=0)

In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19317 entries, 0 to 19363
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   district_id           19317 non-null  int64  
 1   region_id             19317 non-null  int64  
 2   bodyType              19317 non-null  object 
 3   brand                 19317 non-null  object 
 4   model                 19317 non-null  object 
 5   color                 19317 non-null  object 
 6   fuelType              19317 non-null  object 
 7   modelDate             19317 non-null  int64  
 8   name                  19317 non-null  object 
 9   numberOfDoors         19317 non-null  int64  
 10  productionDate        19317 non-null  int64  
 11  vehicleConfiguration  19317 non-null  object 
 12  engineDisplacement    19317 non-null  object 
 13  enginePower           19317 non-null  int64  
 14  description           18726 non-null  object 
 15  mileage            

In [90]:
# Запишем результат парсинга в файл в файл
# name_file = 'auto_ru_BMW_new_districts' + '.csv'
# auto_df.to_csv(name_file, index = False)

In [56]:
Author = 'Arcis'
Data = '10.09.2020'

# Спасибо за помощь в парсинге выражаю Timur-у Abdualimov-у
# а также автору видео https://youtu.be/cQPcEZoxvi0

In [109]:
name_file = 'auto_ru_15092020_BMW_6_districts_v_02' + '.csv'
df.to_csv(name_file, index = False)